# Dataset DataAugmentation

# Importando módulos 

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from PIL import Image
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras import utils as np_utils
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans
from skimage import img_as_ubyte
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
import random
import os

print('Done')

Done


In [ ]:
#funções para data augmentation

def random_rotation(image_array: ndarray):
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree,mode='edge')

def random_noise(image_array: ndarray):
    return sk.util.random_noise(image_array, mode='gaussian')


In [ ]:
#arquivo de labels
arq = open('C:/Users/Andre/Desktop/teste_256_b/train.txt', 'w')

for z in range(len(x_train)):
    #aplicando transformação de ruído
    transformed_image = random_rotation(x_train[z])
    transformed_image2 = random_noise(transformed_image)

    # define a name for our new file
    new_file_path = '%s/augmented_image2_%s.jpg' % ('C:/Users/Andre/Desktop/teste_256_b/data/',z+1)

    # write image to the disk
    io.imsave(new_file_path, transformed_image2)

    #escrevendo no arquivo de labels
    texto = arq.write('augmented_image2_' + str(z+1) + '.jpg' + ' ' + str(y_train[z][0]))
    texto = arq.write('\n')


In [ ]:
#plotting the image
#plt.imshow(transformed_image2)
#plt.axis("off")
#plt.show()

# Definindo caminho dos arquivos

In [3]:
## Caminho
#o_path = 'C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/meses/'
o_path = 'C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/'
print('Done')

Done


# Definindo algumas variáveis

In [4]:
## Classes
num_classes = 12

## Batch Size
batch_size = 64

## Epochs
n_epochs = 64

## Train and Test files
train_file = o_path + 'train.txt'
test_file = o_path + 'test.txt'

## Input Image Dimension
img_rows, img_cols = 32, 32

print('Done')

Done


# Funções para ler e preparar a base de dados

In [5]:
## Resize

def resize_data(data, size, convert):

	if convert:
		data_upscaled = np.zeros((data.shape[0], size[0], size[1], 3))
	else:
		data_upscaled = np.zeros((data.shape[0], size[0], size[1]))
	for i, img in enumerate(data):
		large_img = cv2.resize(img, dsize=(size[1], size[0]), interpolation=cv2.INTER_CUBIC)
		data_upscaled[i] = large_img

	#print (np.shape(data_upscaled))
	return data_upscaled
  
print('Done')

Done


In [6]:
## Load Images

def load_images(image_paths, convert=False):

	x = []
	y = []
	for image_path in image_paths:

		path, label = image_path.split(' ')
		
		## Image path
		path= o_path + 'data/' + path
		print (path)

		if convert:
			image_pil = Image.open(path).convert('RGB') 
		else:
			image_pil = Image.open(path).convert('L')

		img = np.array(image_pil, dtype=np.uint8)

		x.append(img)
		y.append([int(label)])

	x = np.array(x)
	y = np.array(y)

	if np.min(y) != 0: 
		y = y-1

	return x, y

print('Done')

Done


In [7]:
## Load Dataset

def load_dataset(train_file, test_file, resize, convert=False, size=(224,224)):

	arq = open(train_file, 'r')
	texto = arq.read()
	train_paths = texto.split('\n')
	
	print ('Size:', size)

	#train_paths.remove('') # Remove empty lines
	#train_paths.sort()

	print ("Loading training set...")
	x_train, y_train = load_images(train_paths, convert)
 
	arq = open(test_file, 'r')
	texto = arq.read()
	test_paths = texto.split('\n')

	#test_paths.remove('') # Remove empty lines
	#test_paths.sort()
 
	print ("Loading testing set...")
	x_test, y_test = load_images(test_paths, convert)

	if resize:
		print ("Resizing images...")
		x_train = resize_data(x_train, size, convert)
		x_test = resize_data(x_test, size, convert)

	if not convert:
		x_train = x_train.reshape(x_train.shape[0], size[0], size[1], 1)
		x_test = x_test.reshape(x_test.shape[0], size[0], size[1], 1)

	print (np.shape(x_train))
	return (x_train, y_train), (x_test, y_test)
 
print('Done')

Done


# **1. Carregando as bases de treino e teste**

In [8]:
print ("Loading database...")

## Gray Scale
input_shape = (img_rows, img_cols, 1)
(x_train, y_train), (x_test, y_test) = load_dataset(train_file, test_file, resize=True, convert=False, size=(img_rows, img_cols))

## RGB
#input_shape = (img_rows, img_cols, 3)
#(x_train, y_train), (x_test, y_test) = load_dataset(train_file, test_file, resize=True, convert=True, size=(img_rows, img_cols))

## Save for the confusion matrix
label = []
for i in range(len(x_test)):
	label.append(y_test[i][0])

## Normalize images
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
#print ('\n','x_train shape:', x_train.shape)

print ('\n',x_train.shape[0], 'train samples')
print ('\n',x_test.shape[0], 'test samples')



Loading database...
Size: (32, 32)
Loading training set...
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/AD0071.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/AD0072.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/AD0073.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/AD0074.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/AD0075.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/AD0076.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/AD0077.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/AD0078.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/AD0079.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/AD0080.jpg
C:/Users/Andre/Deskto

<ipython-input-6-da8a954948b8>:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x = np.array(x)


C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/td0403.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/td0404.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/td0405.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/td0406.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/td0407.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/td0408.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/td0409.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/td0410.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/td0411.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data/td0412.jpg
C:/Users/Andre/Desktop/M_2021_1/AM/Lab3/bases_lab3_antigas/data_AUGMENTED64/data

In [9]:
########################################
## Convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)
print('Done')

Done


In [10]:
# salvando os arrays para trainar no colab
np.save('C:/Users/Andre/Desktop/base_augmented_32/x_train.npy', x_train) 
np.save('C:/Users/Andre/Desktop/base_augmented_32/y_train.npy', y_train) 
np.save('C:/Users/Andre/Desktop/base_augmented_32/x_test.npy', x_test)
np.save('C:/Users/Andre/Desktop/base_augmented_32/y_test.npy', y_test) 
np.save('C:/Users/Andre/Desktop/base_augmented_32/label.npy',label)
print('Done')

Done


In [ ]:
#para carregar os arquivos do numpy no colab:
#SAVED_ARRAY = np.load('NAME_OF_ARRAY.npy')